# Download NMR Dart survey data

**Author:** Logan Schmidt

**Created:** 2019-06-07

**Last Updated:** 2019-06-07

**Purpose:**

This script was created as part of the effort to produce the **Schmidt, Rempe 2019 Water Content** manuscript. This script accesses the NMR Dart survey data saved on the **DRempe Drive** on the JSG server and imports, processes, and saves the data as a single master csv file.

In [22]:
import tkinter as tk
from tkinter import filedialog
import pandas as pd
import numpy as np
import os
import glob
import re

In [23]:
# select processed NMR folder to get directory
root = tk.Tk()
importDir = filedialog.askdirectory(title = 'Select Processed NMR Folder')
root.mainloop()
if len(importDir) > 0:
    print('You chose: %s' % importDir)

You chose: /Volumes/DRempe Drive/Logan Schmidt/NMR/DATA/ERCZO/Processed_All/Processed_Survey/well_007_2018_08_03


In [25]:
# get directories of .txt files containing datadx
fileNames = sorted(glob.glob(importDir + '/*.txt'))

In [69]:
############################################################
# Import 1D vectors as a dataframe, df1, with added metadata 
############################################################

# convert the *_1Dvectors_uniform.txt file into a dataframe
df1 = pd.read_csv(fileNames[1], sep = '\s+')

# disect filName to get wellID and year-month-day
exps0 = re.findall('/(\w*)', fileNames[1])
exps1 = re.findall('(\w*?)_', exps0[-1])
wellID = exps1[1]
year = exps1[2]
month = exps1[3]
day = exps1[4]
if day == '1Dvectors':
    day = '01'
datetime = pd.to_datetime(year + month + day)

# add wellID and date vectors
df1.insert(0, 'wellID', wellID, True)
df1.insert(1, 'date', datetime, True)

# df1.head()

In [71]:
df1.set_index([df1['wellID'], df1['date'], df1['depth']], inplace = True)

df1.head()

wellID       date  depth    totalf     clayf  \
wellID date       depth                                                
007    2018-08-03 1.5      007 2018-08-03    1.5  0.079407  0.042501   
                  2.0      007 2018-08-03    2.0  0.147774  0.127677   
                  2.5      007 2018-08-03    2.5  0.112298  0.078419   
                  3.0      007 2018-08-03    3.0  0.102325  0.076267   
                  3.5      007 2018-08-03    3.5  0.114206  0.090612   

                             capf     freef      mlT2      Ksdr           Ktc  \
wellID date       depth                                                         
007    2018-08-03 1.5    0.036906  0.000000  0.002889  0.005899  0.000000e+00   
                  2.0    0.015110  0.004987  0.001903  0.004761  5.000000e-10   
                  2.5    0.033879  0.000000  0.002179  0.004746  0.000000e+00   
                  3.0    0.025321  0.000737  0.002107  0.004043  0.000000e+00   
                  3.5    0.014810  0.008784  0.002229  0.005051  1.000000e-09   

                             Ksoe      Tsdr           Ttc      Tsoe  \
wellID date       depth                                               
007    2018-08-03 1.5    0.001067  0.002950  0.000000e+00  0.000534   
                  2.0    0.041920  0.005330  2.000000e-10  0.021494   
                  2.5    0.003744  0.007703  2.000000e-10  0.023366   
                  3.0    0.005711  0.009724  2.000000e-10  0.026221   
                  3.5    0.009191  0.012250  7.000000e-10  0.030817   

                                  soe     noise  
wellID date       depth                          
007    2018-08-03 1.5    7.069000e-07  3.128624  
                  2.0    1.525780e-05  3.114880  
                  2.5    2.479300e-06  3.195702  
                  3.0    2.251500e-06  3.039879  
                  3.5    5.837500e-06  2.896534

In [50]:
####################################################################################
# Import T2 distribution data as a dataframe, df2, and join to master dataframe, df0 
####################################################################################

# convert the *_T2_dist_uniform.txt file into a dataframe
df2 = pd.read_csv(fileNames[7],
                  sep = '\t',
                  header = None)
# df2a = pd.read_csv(fileNames[7], 
#                    sep = '\t', 
#                    names = ['depth (m)'], 
#                    usecols = [0])
# df2b = pd.read_csv(fileNames[7],
#                   sep = '\t',
#                   header = None,
#                   usecols = list(range(1,101)) 
#                   )

df2.head()

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,1.5,0.001617,0.001655,0.001731,0.001842,0.001984,0.002150,0.002337,0.002536,0.002742,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2.0,0.010253,0.010197,0.010084,0.009915,0.009690,0.009408,0.009070,0.008679,0.008237,...,0.000089,0.000113,0.000137,0.000161,0.000184,0.000204,0.000222,0.000236,0.000246,0.000251
2,2.5,0.004773,0.004788,0.004815,0.004853,0.004898,0.004946,0.004993,0.005033,0.005061,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,3.0,0.005517,0.005501,0.005469,0.005420,0.005354,0.005268,0.005162,0.005036,0.004889,...,0.000003,0.000011,0.000024,0.000042,0.000063,0.000085,0.000105,0.000123,0.000136,0.000143
4,3.5,0.006621,0.006607,0.006578,0.006530,0.006463,0.006370,0.006250,0.006099,0.005913,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [47]:
df = pd.DataFrame({'month': [1, 4, 7, 10],
                    'year': [2012, 2014, 2013, 2014],
                    'sale': [55, 40, 84, 31]})
df.set_index(['year'], drop = True, append = True)

df

,month,year,sale
0,1,2012,55
1,4,2014,40
2,7,2013,84
3,10,2014,31
